# Databricks Unity Catalog: End-to-End Data Governance & Security Lab
**This notebook provides a step-by-step lab for data governance and security using Unity Catalog in Databricks.**

**Key tasks:**
- Catalog/schema/table setup
- Data migration
- Encryption awareness
- Access control (including dynamic views)
- Data lifecycle management
- Marketplace integration
- Audit and validation

---
## Prerequisites
- Unity Catalog must be enabled.
- You need sufficient permissions (admin or equivalent for governance tasks).
- Sample data should be available in DBFS or legacy databases.
---

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS spark_catalog.legacy_db;


In [0]:
# sales.csv
df_sales = spark.read.csv('/FileStore/data/sales.csv', header=True, inferSchema=True)
df_sales.write.mode('overwrite').saveAsTable('spark_catalog.legacy_db.sales')

# customer_details.csv
df_cust = spark.read.csv('/FileStore/data/customer_details.csv', header=True, inferSchema=True)
df_cust.write.mode('overwrite').saveAsTable('spark_catalog.legacy_db.customer_details')

# product_catalog.csv
df_prod = spark.read.csv('/FileStore/data/product_catalog.csv', header=True, inferSchema=True)
df_prod.write.mode('overwrite').saveAsTable('spark_catalog.legacy_db.product_catalog')


In [0]:
%sql
SELECT * FROM spark_catalog.legacy_db.sales LIMIT 5;


## 1. Create Catalog, Schema, and Table
**Why:** Establishes data organization and logical boundaries for governance.

**Action:** Create a catalog, then a schema, then a managed table (example uses DELTA format).

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS legacymigration
  MANAGED LOCATION 'abfss://data@deassociateadls.dfs.core.windows.net/uc/legacymigration'
  COMMENT 'UC Demo Catalog for governance lab';


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS legacymigration.raw;


## 2. Migrate Existing Table/Data to Unity Catalog
**Why:** Bring legacy tables under Unity Catalog for unified governance and security.

**Action:** List legacy tables, then use CTAS (Create Table As Select) to migrate.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS legacymigration.raw.sales AS
SELECT * FROM spark_catalog.legacy_db.sales;

CREATE TABLE IF NOT EXISTS legacymigration.raw.customer_details AS
SELECT * FROM spark_catalog.legacy_db.customer_details;

CREATE TABLE IF NOT EXISTS legacymigration.raw.product_catalog AS
SELECT * FROM spark_catalog.legacy_db.product_catalog;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM legacymigration.raw.sales LIMIT 5;


SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerID,Item,Quantity,UnitPrice,TaxAmount
SO20000,2,2024-08-24,CUST2373,Running Shorts,4,278.63,55.73
SO20001,2,2024-10-06,CUST2779,Vacuum Cleaner,4,111.58,22.32
SO20002,2,2024-08-02,CUST2732,Tennis Racket,6,152.03,45.61
SO20003,3,2024-09-21,CUST1815,Children's Book,3,6.62,0.99
SO20004,2,2025-03-18,CUST2913,Doll,7,31.13,10.9


In [0]:
%sql
SHOW DATABASES;

databaseName
default
legacy_db


In [0]:
# Optional: List legacy tables
display(spark.sql('SHOW TABLES IN spark_catalog.legacy_db'))

database,tableName,isTemporary
legacy_db,customer_details,false
legacy_db,product_catalog,false
legacy_db,sales,false
,_sqldf,true


In [0]:
# Optional: List UC tables
display(spark.sql('SHOW TABLES IN legacymigration.raw'))

database,tableName,isTemporary
raw,customer_details,false
raw,product_catalog,false
raw,sales,false
,_sqldf,true


## 3. Encryption Awareness
**Why:** All data in Unity Catalog is encrypted at rest and in transit. Customer-managed keys (CMK) are configured by admins at the workspace/cloud level.**

**Action:** No code required here for most users; verify via Admin Console if needed.

## 4. Apply Security Policies & Fine-Grained Access Control
**Why:** Ensures only authorized users/groups can access or manipulate data, fulfilling compliance and privacy requirements.**

**Action:** Grant usage, grant table SELECT, and build a dynamic view for row-level security.

In [0]:
%sql
/* Grant catalog and schema usage to a group (replace `data_engineers` with your real group)*/
GRANT USAGE ON CATALOG legacymigration TO `bi_group`;
GRANT USAGE ON SCHEMA legacymigration.raw TO `bi_group`;

GRANT SELECT ON TABLE legacymigration.raw.sales TO `analysts`;
GRANT SELECT ON TABLE legacymigration.raw.customer_details TO `analysts`;
GRANT SELECT ON TABLE legacymigration.raw.product_catalog TO `analysts`;


In [0]:
%sql
CREATE OR REPLACE VIEW legacymigration.raw.customer_details_masked AS
SELECT
  *,
  CASE WHEN is_member('admins') THEN EmailAddress ELSE '***MASKED***' END AS masked_email
FROM legacymigration.raw.customer_details;

GRANT SELECT ON VIEW legacymigration.raw.customer_details_masked TO `account users`;


## 5. Data Lifecycle Management
**Why:** Automates data retention and cleanup for compliance and cost control.**

**Action:** Set a Delta retention policy and use VACUUM to clean up old files.

In [0]:
%sql
ALTER TABLE legacymigration.raw.sales SET TBLPROPERTIES (
  'delta.deletedFileRetentionDuration' = 'interval 30 days'
);

VACUUM legacymigration.raw.sales RETAIN 720 HOURS;


path
abfss://data@deassociateadls.dfs.core.windows.net/uc/legacymigration/__unitystorage/catalogs/448b6790-3f53-4c3e-bcff-19f997219f30/tables/7017ab19-de6a-459d-a36d-726e67686016


## 6. Audit Logs & Validation
**Why:** Enables compliance monitoring, troubleshooting, and validation of security settings.**

**Action:** Audit logs are accessed via Admin Console or your cloud provider. Validate row-level security:

In [0]:
# Query as analyst (or test with different users)
display(spark.sql('SELECT * FROM legacymigration.raw.customer_details_masked'))

CustomerID,CustomerName,EmailAddress,PhoneNumber,Address,City,State,PostalCode,Country,masked_email
CUST1000,Chelsea Dennis,chelsea.dennis@example.com,+1-888-897-8154x5654,"930 Taylor Green Apt. 725, Kramerstad, AZ 77359",Lisaville,DE,2021,Moldova,chelsea.dennis@example.com
CUST1001,Jane Thomas,jane.thomas@example.com,6187198217,"7233 Jones Falls Suite 395, Welchborough, AR 74192",North Lisa,IA,18234,Djibouti,jane.thomas@example.com
CUST1002,Robin Adams,robin.adams@example.com,331-297-4367,"128 Amy Union, Port Susanfort, SD 52904",Port Alyssa,WV,69487,Niue,robin.adams@example.com
CUST1003,Nicholas Wilson,nicholas.wilson@example.com,+1-660-383-3694x76225,"627 Joe Mount Apt. 316, South Frank, KS 63551",West Mark,OH,44904,El Salvador,nicholas.wilson@example.com
CUST1004,Douglas Freeman,douglas.freeman@example.com,028-230-6853x306,"PSC 6825, Box 6377, APO AE 04725",West Troyville,DE,45581,Seychelles,douglas.freeman@example.com
CUST1005,Eric King,eric.king@example.com,842.723.4691x69876,"21724 Jones Island Apt. 019, Hawkinsberg, OH 46544",New Briannaside,OH,79271,Kyrgyz Republic,eric.king@example.com
CUST1006,Colin Thomas,colin.thomas@example.com,001-703-833-9157x181,"404 David Islands, Smithside, AL 79254",Marthatown,CA,60129,Dominican Republic,colin.thomas@example.com
CUST1007,Dakota Adams,dakota.adams@example.com,+1-163-858-5831x7706,"465 Palmer Village, Chavezport, PA 55675",North Markberg,UT,89972,Niue,dakota.adams@example.com
CUST1008,Amy Bradley,amy.bradley@example.com,(775)471-6742,"55873 Brown Roads, South Erin, KY 25590",Clarkeport,NM,29523,Liberia,amy.bradley@example.com
CUST1009,Teresa Chambers,teresa.chambers@example.com,(363)209-7153x874,"9419 Carson Mews, Garzaburgh, AL 80427",West Shannon,NV,579,Netherlands Antilles,teresa.chambers@example.com


## 7. Clean Up (Optional)
**Why:** Remove resources to save cost or reset lab for the next run.**

**Action:** Uncomment and run as needed.

In [0]:
%sql
DROP VIEW IF EXISTS legacymigration.raw.customer_details_masked;
DROP TABLE IF EXISTS legacymigration.raw.sales;
DROP TABLE IF EXISTS legacymigration.raw.customer_details;
DROP TABLE IF EXISTS legacymigration.raw.product_catalog;
DROP SCHEMA IF EXISTS legacymigration.raw CASCADE;
DROP CATALOG IF EXISTS legacymigration CASCADE;

---
### End of Databricks Unity Catalog Governance Lab
- Review audit logs for compliance.
- Validate access as different users.


## Reading the Data from ADLS with Connector

In [0]:
sales_df = spark.read.format("csv").option("header", "true").load("abfss://data@deassociateadls.dfs.core.windows.net/mycatalog/files/sales.csv")
display(sales_df)

SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerID,Item,Quantity,UnitPrice,TaxAmount
SO20000,2,2024-08-24,CUST2373,Running Shorts,4,278.63,55.73
SO20001,2,2024-10-06,CUST2779,Vacuum Cleaner,4,111.58,22.32
SO20002,2,2024-08-02,CUST2732,Tennis Racket,6,152.03,45.61
SO20003,3,2024-09-21,CUST1815,Children's Book,3,6.62,0.99
SO20004,2,2025-03-18,CUST2913,Doll,7,31.13,10.9
SO20005,3,2024-12-30,CUST1241,Vitamin Supplements,2,247.52,24.75
SO20006,2,2024-09-17,CUST2392,Pruning Shears,9,307.9,138.56
SO20007,1,2024-08-05,CUST2277,Remote Car,7,6.8,2.38
SO20008,3,2024-07-04,CUST1977,Coffee Maker,2,493.13,49.31
SO20009,4,2025-03-30,CUST1598,Car Vacuum,4,98.41,19.68


Databricks data profile. Run in Databricks to view.